In [271]:
from pandas import read_csv
import pandas as pd
import numpy as np
from jina import Document, DocumentArray

In [272]:
def get_preds(index_path):
    da = DocumentArray.load(index_path)
    da.match(da, exclude_self=True)
    
    preds =[]
    for d in da:
        if d.matches[0].tags.get("known_label") == 0:
            preds.append("Benign")
        else:
            preds.append("Attack")
    return preds
    

## in app

In [273]:
da = DocumentArray.load("index")
da.summary()

╭────────────────── Documents Summary ───────────────────╮
│                                                        │
│   Length                 15101                         │
│   Homogenous Documents   True                          │
│   Common Attributes      ('id', 'tags', 'embedding')   │
│                                                        │
╰────────────────────────────────────────────────────────╯
╭────────────────────── Attributes Summary ───────────────────────╮
│                                                                 │
│   Attribute   Data type      #Unique values   Has empty value   │
│  ─────────────────────────────────────────────────────────────  │
│   embedding   ('ndarray',)   15101            False             │
│   id          ('str',)       15101            False             │
│   tags        ('dict',)      15101            False             │
│                                                                 │
╰─────────────────────────────────────────────────────────────────╯

In [274]:
da_df = da.to_dataframe()
da_df.head(2)

,id,parent_id,granularity,adjacency,blob,tensor,mime_type,text,weight,uri,tags,offset,location,embedding,modality,evaluations,scores,chunks,matches
0,a48274ca3bc05873926acf05cfc76f82,None,None,None,None,None,None,None,None,None,"{'dt': '2018-01-20 13:30:11', 'known_label': 0.0}",None,None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",None,None,None,None,None
1,8d49cfeadb60ba8d1cc56cdce1c908e2,None,None,None,None,None,None,None,None,None,"{'dt': '2018-01-20 13:30:18', 'known_label': 0.0}",None,None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",None,None,None,None,None


In [275]:
da_df = da_df[["id", "tags", "embedding"]]
da_df.head(2)

,id,tags,embedding
0,a48274ca3bc05873926acf05cfc76f82,"{'dt': '2018-01-20 13:30:11', 'known_label': 0.0}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,8d49cfeadb60ba8d1cc56cdce1c908e2,"{'dt': '2018-01-20 13:30:18', 'known_label': 0.0}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [276]:
da_df["id"] = da_df["id"].apply(lambda x: x[:8])
da_df["datetime"] = da_df["tags"].apply(lambda x: x.get("dt"))
da_df["datetime"]= pd.to_datetime(da_df["datetime"])
da_df["known_label"] = da_df["tags"].apply(lambda x: x.get("known_label"))
da_df["known_label"] = da_df["known_label"].map(lambda x: "Benign" if x == 0.0 else "Attack")
da_df["predicted"] = get_preds("index")
da_df['is_wrong'] = da_df.apply(lambda x: x['predicted'] != x['known_label'], axis=1)

In [277]:
da_df.head(2)

,id,tags,embedding,datetime,known_label,predicted,is_wrong
0,a48274ca,"{'dt': '2018-01-20 13:30:11', 'known_label': 0.0}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2018-01-20 13:30:11,Benign,Benign,False
1,8d49cfea,"{'dt': '2018-01-20 13:30:18', 'known_label': 0.0}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2018-01-20 13:30:18,Benign,Benign,False


In [278]:
da_df.tail(2)

,id,tags,embedding,datetime,known_label,predicted,is_wrong
15099,370533ef,"{'dt': '2018-01-23 04:49:58', 'known_label': 0.0}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2018-01-23 04:49:58,Benign,Benign,False
15100,9d15f05e,"{'dt': '2018-01-23 04:50:27', 'known_label': 0.0}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2018-01-23 04:50:27,Benign,Benign,False


In [279]:
import plotly.express as px
import plotly.graph_objects as go

In [280]:
da_df.head(2)

,id,tags,embedding,datetime,known_label,predicted,is_wrong
0,a48274ca,"{'dt': '2018-01-20 13:30:11', 'known_label': 0.0}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2018-01-20 13:30:11,Benign,Benign,False
1,8d49cfea,"{'dt': '2018-01-20 13:30:18', 'known_label': 0.0}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2018-01-20 13:30:18,Benign,Benign,False


In [281]:
da_df["datetime"] = pd.to_datetime(da_df["datetime"])
# da_df["counts"] = da_df["counts"].value_counts()
da_df["is_wrong"].value_counts().head(2)

False    15067
True        34
Name: is_wrong, dtype: int64

In [282]:
da_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15101 entries, 0 to 15100
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           15101 non-null  object        
 1   tags         15101 non-null  object        
 2   embedding    15101 non-null  object        
 3   datetime     15101 non-null  datetime64[ns]
 4   known_label  15101 non-null  object        
 5   predicted    15101 non-null  object        
 6   is_wrong     15101 non-null  bool          
dtypes: bool(1), datetime64[ns](1), object(5)
memory usage: 722.7+ KB


In [283]:
da_df.set_index("datetime", inplace=True)

In [284]:
da_df[da_df.index.minute == 12].tail()

,id,tags,embedding,known_label,predicted,is_wrong
datetime,,,,,,
2018-01-23 03:12:18,53d96f1d,"{'known_label': 0.0, 'dt': '2018-01-23 03:12:18'}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Benign,Benign,False
2018-01-23 04:12:07,7fff7195,"{'known_label': 0.0, 'dt': '2018-01-23 04:12:07'}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Benign,Benign,False
2018-01-23 04:12:14,6f6f2e0a,"{'dt': '2018-01-23 04:12:14', 'known_label': 0.0}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Benign,Benign,False
2018-01-23 04:12:28,66681d1b,"{'known_label': 0.0, 'dt': '2018-01-23 04:12:28'}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Benign,Benign,False
2018-01-23 04:12:32,f6037462,"{'known_label': 0.0, 'dt': '2018-01-23 04:12:32'}","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Benign,Benign,False


In [ ]:
fig_scat = da_df[da_df["predicted"] == "Attack"]
fig_scat

In [294]:
fig_scat = da_df[da_df["predicted"] == "Attack"]

# produce figure
fig = px.scatter(da_df, x=da_df.index, y=da_df["predicted"])

# show figure
fig.show()